In [1]:
%run /spark-data/CRM/utilities/common_utility.ipynb

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/14 13:56:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/14 13:56:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/09/14 13:56:05 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


# Initialize Spark Session

In [2]:
spark = initialize_spark_session("Sales Team Cleaning")

24/09/14 13:56:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


# Logs Configuration

In [3]:
log_file_path = 'logs/sales_team_cleaning.log'
logger = initialize_logger(log_file_path)

logger.info("Logger initialized with dynamic path!")

2024-09-14 13:56:07,268 - logger - INFO - Logger initialized with dynamic path!


# Dataset Load

In [4]:
sales_team_path = "/spark-data/CRM/Dataset/sales_team.csv"
sales_team_df = load_data_files(sales_team_path)
display_dataframes(sales_team_df)

2024-09-14 13:56:11,773 - logger - INFO - Displayed first 5 records of Spark DataFrame.


+------------------------------------+-----------------+-------------+------------+--------------+
|Sales_Rep_ID                        |Name             |Region       |Sales_Target|Sales_Achieved|
+------------------------------------+-----------------+-------------+------------+--------------+
|0437b05a-9628-43f9-ac07-0b9a0dc96dcd|Brittany Taylor  |California   |41135       |14037.0       |
|4daeb6af-d7e9-4f99-91b3-6c912f45b740|Mitchell Williams|New Hampshire|32996       |21461.0       |
|f243144e-485f-4382-81ef-2a9a3c63f172|John Terry       |Kansas       |10385       |NULL          |
|9c44ee81-8254-45e1-af23-a4608ceb126c|Carolyn Miller   |Arizona      |23754       |17149.0       |
|3e97b5d8-933a-4860-bce7-2398af6c5613|Antonio Sparks   |Washington   |27101       |36413.0       |
+------------------------------------+-----------------+-------------+------------+--------------+
only showing top 5 rows



# Data Preprocessing

In [5]:
# Step 1: Identify Missing Values
logger.info("Step 1: Identifying missing values in each column before filling them...")
missing_values_before = count_missing_values(sales_team_df)
missing_values_before.show()

# Step 2: Handle Missing Values
logger.info("Step 2: Handling missing values by filling with averages...")
# Calculate average Sales_Achieved
avg_sales_achieved = sales_team_df.select(avg(col("Sales_Achieved"))).first()[0]
print(f"Average value of sales achieved column is {avg_sales_achieved}")

# Fill missing Sales_Achieved with the average
sales_team_cleaned_df = fill_missing_values(sales_team_df,{"Sales_Achieved": round(avg_sales_achieved,2)})

# Step 3: Check for Duplicate Values
logger.info("Step 3: Checking for duplicate records based on 'Sales_Rep_ID'...")
duplicate_count_before = count_duplicates_per_column(sales_team_cleaned_df)

# Step 4: Drop Duplicates if they exist
logger.info("Step 4: Dropping duplicate records based on 'Sales_Rep_ID'...")
sales_team_cleaned_df = drop_duplicates(sales_team_cleaned_df, "Sales_Rep_ID")

# Step 6: Standardize Formats
logger.info("Step 5: Standardizing the format of 'Name' and 'Region' columns...")
sales_team_cleaned_df = capitalize_columns(sales_team_cleaned_df, [ "Name", "Region"])
print("Completed.")

# Step 7: Cross-verification of missing values in each column after filling them
logger.info("Step 6: Identifying missing values in each column after filling them...")
missing_values_after = count_missing_values(sales_team_cleaned_df)
missing_values_after.show()

# Step 8: Display the cleaned 'sales_team_df' DataFrame
logger.info("Step 7: Displaying the cleaned 'sales_team_df' DataFrame...")
sales_team_cleaned_df.show(5,truncate=False)

# Step 9: Save the cleaned data to a new CSV
logger.info("Step 8: Saving the cleaned data to 'cleaned_sales_team.csv'...")
# save_df_to_csv(sales_team_cleaned_df, "Cleaned_data/cleaned_sales_team.csv")

# Display the count of records after phone number processing
record_count_after_cleaning = sales_team_cleaned_df.count()
print(f"Number of records after cleaning: {record_count_after_cleaning}")
logger.info("Data cleaning and export completed successfully.")

2024-09-14 13:56:11,787 - logger - INFO - Step 1: Identifying missing values in each column before filling them...
2024-09-14 13:56:12,617 - logger - INFO - Step 2: Handling missing values by filling with averages...


+------------+----+------+------------+--------------+
|Sales_Rep_ID|Name|Region|Sales_Target|Sales_Achieved|
+------------+----+------+------------+--------------+
|           0|   0|     0|           0|             5|
+------------+----+------+------------+--------------+



2024-09-14 13:56:12,940 - logger - INFO - Step 3: Checking for duplicate records based on 'Sales_Rep_ID'...


Average value of sales achieved column is 21912.425531914894


2024-09-14 13:56:15,463 - logger - INFO - Step 4: Dropping duplicate records based on 'Sales_Rep_ID'...
2024-09-14 13:56:15,859 - logger - INFO - Number of duplicate records before dropping: 2
2024-09-14 13:56:16,382 - logger - INFO - Number of duplicate records after dropping: 0
2024-09-14 13:56:16,385 - logger - INFO - Step 5: Standardizing the format of 'Name' and 'Region' columns...
2024-09-14 13:56:16,432 - logger - INFO - Step 6: Identifying missing values in each column after filling them...


...

+------------+----+------+------------+--------------+
|Sales_Rep_ID|Name|Region|Sales_Target|Sales_Achieved|
+------------+----+------+------------+--------------+
|           0|   0|     0|           0|             0|
+------------+----+------+------------+--------------+



2024-09-14 13:56:17,651 - logger - INFO - Step 8: Saving the cleaned data to 'cleaned_sales_team.csv'...


+------------------------------------+---------------------+------------+------------+--------------+
|Sales_Rep_ID                        |Name                 |Region      |Sales_Target|Sales_Achieved|
+------------------------------------+---------------------+------------+------------+--------------+
|02acdbab-d149-4053-8c7a-871ba76f003e|Daniel Barber        |Arizona     |13651       |7296.0        |
|0437b05a-9628-43f9-ac07-0b9a0dc96dcd|Brittany Taylor      |California  |41135       |14037.0       |
|0468e32e-644b-491f-a3e1-741306f2c3f2|Dr. Wayne Spencer Dvm|Pennsylvania|15593       |37653.0       |
|05cc8513-8a8a-4598-862a-218d1093ae26|Pamela Pennington    |Idaho       |38399       |11569.0       |
|06a9a95f-ae20-44ae-ad5d-d4bdf5f121d0|Sean Snyder          |Illinois    |19165       |11748.0       |
+------------------------------------+---------------------+------------+------------+--------------+
only showing top 5 rows



2024-09-14 13:56:17,956 - logger - INFO - Data cleaning and export completed successfully.


Number of records after cleaning: 50


In [6]:
spark.stop()